In [ ]:
import paramiko
import io
import os

from dotenv import load_dotenv

load_dotenv()

# pip install paramiko
# pip install python-dotenv
# hostname = ""
# port = 22
# username = ""
# password = ""

hostname = os.getenv("SFTP_HOST")
port = int(os.getenv("SFTP_PORT"))
username = os.getenv("SFTP_USER")
password = os.getenv("SFTP_PASSWORD")

if not password:
    raise ValueError("❌ SFTP_PASSWORD nicht in .env definiert!")

print(f"🔄 Verbinde zu {hostname}:{port} als {username}...")

try:
    transport = paramiko.Transport((hostname, port))
    transport.start_client()
    transport.auth_password(username=username, password=password)

    if not transport.is_authenticated():
        raise paramiko.AuthenticationException("Auth failed")

    print("✅ SFTP-Verbindung erfolgreich!")

    sftp = paramiko.SFTPClient.from_transport(transport)
    sftp.chdir("out")

    # === LV.ZIP HERUNTERLADEN ===
    print("📥 Lade lv.zip...")
    lv_zip_data = io.BytesIO()
    sftp.getfo("lv.zip", lv_zip_data)
    lv_zip_data.seek(0)
    print(f"  ✓ {len(lv_zip_data.getvalue()) / (1024**2):.1f} MB")

    # === AKI.ZIP HERUNTERLADEN ===
    print("📥 Lade aki.zip...")
    aki_zip_data = io.BytesIO()
    sftp.getfo("aki.zip", aki_zip_data)
    aki_zip_data.seek(0)
    print(f"  ✓ {len(aki_zip_data.getvalue()) / (1024**2):.1f} MB")

    sftp.close()
    transport.close()
    print("✅ Download abgeschlossen!")

except paramiko.AuthenticationException as e:
    raise ValueError(f"❌ Authentifizierung fehlgeschlagen: {e}")
except Exception as e:
    raise ConnectionError(f"❌ SFTP-Fehler: {e}")

🔄 Verbinde zu ftp.bruder-gruppe.de:22 als hochschuleog...
✅ SFTP-Verbindung erfolgreich!
📥 Lade lv.zip...
  ✓ 440.9 MB
📥 Lade aki.zip...


In [ ]:
import zipfile
import subprocess

os.makedirs("./data", exist_ok=True)

# === LV.ZIP ENTPACKEN ===
print("📦 Entpacke lv.zip...")
try:
    with zipfile.ZipFile(lv_zip_data, 'r', allowZip64=True) as z:
        z.extractall("./data")
    print("  ✓ Erfolgreich entpackt")
except NotImplementedError:
    print("  ⚠️ ZIP-Kompression nicht unterstützt, nutze Kommandozeile...")
    lv_zip_data.seek(0)
    with open("/tmp/lv.zip", "wb") as f:
        f.write(lv_zip_data.read())
    subprocess.run(["unzip", "-o", "/tmp/lv.zip", "-d", "./data"], check=True)
    print("  ✓ Mit unzip entpackt")

# === AKI.ZIP ENTPACKEN ===
print("📦 Entpacke aki.zip...")
try:
    with zipfile.ZipFile(aki_zip_data, 'r', allowZip64=True) as z:
        z.extractall("./data")
    print("  ✓ Erfolgreich entpackt")
except NotImplementedError:
    print("  ⚠️ ZIP-Kompression nicht unterstützt, nutze Kommandozeile...")
    aki_zip_data.seek(0)
    with open("/tmp/aki.zip", "wb") as f:
        f.write(aki_zip_data.read())
    subprocess.run(["unzip", "-o", "/tmp/aki.zip", "-d", "./data"], check=True)
    print("  ✓ Mit unzip entpackt")

print("✅ Alle Dateien entpackt!")

In [ ]:
import pandas as pd

# === DATAFRAMES LADEN ===
dataframes = {}

print("\n📊 Lade DataFrames...")

# LV DataFrame
lv_path = "data/lv/lv.csv"
if os.path.exists(lv_path):
    dataframes['lv'] = pd.read_csv(lv_path, low_memory=False)
    print(f"  ✓ lv.csv: {len(dataframes['lv'])} Zeilen")
else:
    print(f"  ❌ {lv_path} nicht gefunden!")

# ART DataFrame
art_path = "data/aki/art.csv"
if os.path.exists(art_path):
    dataframes['art'] = pd.read_csv(art_path, low_memory=False)
    print(f"  ✓ art.csv: {len(dataframes['art'])} Zeilen")
else:
    print(f"  ❌ {art_path} nicht gefunden!")

print(f"\n🎉 Fertig! {len(dataframes)} Datensätze geladen.")
print(f"   Verfügbare DataFrames: {list(dataframes.keys())}")

In [ ]:
# Setup & Normalisierung
import os
import re
from dotenv import load_dotenv

load_dotenv()

LV_OUT_PATH = os.getenv("LV_OUT_PATH", "data/lv_clean.csv")
NROWS = int(os.getenv("LV_NROWS", "0")) or None

if 'lv' in dataframes:
    df = dataframes['lv'].copy()
else:
    raise ValueError("'lv' nicht in geladenen Daten gefunden!")

required_cols = {"id", "oz_bez"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Fehlende Spalten: {missing}")

TRIGGER_PATTERNS = [
    r"\bwie\s+(?:oben|vor|zuvor|zuletzt)\b",
    r"\bw(?:ie)?\s*o\b",
    r"\bdito\b",
    r"\bentspr\.\s*oben\b",
    r"\bgleich\s+wie\b",
    r"\bwie\s+beschr(?:ieben|.)\b",
    r"\bwie\s+pos(?:ition)?\s*\d+\b",
    r"\bwie\s+z(?:ur|u)\s+pos\b",
    r"\bdesgleichen\b",
]
trigger_re = re.compile("|".join(f"(?:{p})" for p in TRIGGER_PATTERNS), flags=re.IGNORECASE)

def has_trigger(text: str) -> bool:
    return bool(trigger_re.search(str(text)))

# Neue Spalten zum Inspizieren
df["Referenz"] = ""
df["Original"] = ""

print("✓ Normalisierung abgeschlossen")
print(f"  Arbeite mit {len(df)} Zeilen aus 'lv'")

In [ ]:
# Referenzen auflösen & Speichern

first_by_id = df.sort_values("id").groupby("id", as_index=True)["oz_bez"].first()

def resolve_reference(current_id: int):
    search_id = current_id - 1
    while search_id in first_by_id.index:
        t = first_by_id.loc[search_id]
        if not has_trigger(t):
            return t
        search_id -= 1
    return None

def resolve_row(row):
    text = row["oz_bez"]
    row["Original"] = text

    if not has_trigger(text):
        row["Referenz"] = ""
        return row

    try:
        cid = int(row["id"])
    except Exception:
        row["Referenz"] = ""
        return row

    base = resolve_reference(cid)
    if base:
        row["Referenz"] = base
        # Bereinige Base und Text SEPARAT vor dem Kombinieren
        base_clean = re.sub(r"\s{2,}", " ", base.strip())
        text_clean = re.sub(r"\s{2,}", " ", text.strip())
        # Kombiniere mit doppeltem Zeilenumbruch
        text = f"Base: {base_clean}\n\nText: {text_clean}"
    else:
        # Wenn kein Base gefunden, nur Text bereinigen
        text = re.sub(r"\s{2,}", " ", text.strip())

    return row


df = df.apply(resolve_row, axis=1)

if "arnr" in df.columns:
    df["arnr"] = df["arnr"].astype(str).str.strip()

os.makedirs(os.path.dirname(LV_OUT_PATH), exist_ok=True)
df.to_csv(LV_OUT_PATH, index=False, encoding="utf-8")

print(f"✓ Fertig: {LV_OUT_PATH}")
print(f"  Spalten: Referenz | Original | oz_bez (kombiniert)")



In [ ]:
# %% md
# ## Bereinigung - Entferne leere/ungültige oz_bez

# %%
# Überprüfe Datenqualität VORHER
print("📊 Datenqualität VOR Bereinigung:")
print(f"  Gesamt-Zeilen: {len(df)}")

# Check für Null-Werte
null_count = df['oz_bez'].isna().sum()
print(f"  NULL-Werte: {null_count}")

# Check für leere Strings
empty_strings = (df['oz_bez'] == '').sum()
print(f"  Leere Strings: {empty_strings}")

# Check für nur Whitespace/Newlines
def is_empty_or_whitespace(text):
    """Prüft ob Text leer, nur Leerzeichen oder nur Newlines ist"""
    if pd.isna(text):
        return True
    text_str = str(text).strip()
    return len(text_str) == 0

whitespace_only = df['oz_bez'].apply(is_empty_or_whitespace).sum()
print(f"  Nur Whitespace/Newlines: {whitespace_only}")

# %%

In [ ]:
# Entferne alle ungültigen Einträge
print("\n🧹 Entferne ungültige Einträge...")

# Filter: Behalte nur Zeilen mit gültigem oz_bez
df_clean = df[~df['oz_bez'].apply(is_empty_or_whitespace)].copy()

removed_count = len(df) - len(df_clean)
print(f"  ✓ Entfernt: {removed_count} Zeilen")
print(f"  ✓ Verbleibend: {len(df_clean)} Zeilen ({len(df_clean)/len(df)*100:.1f}%)")

# Aktualisiere df für weitere Verarbeitung
df = df_clean

# %%

In [ ]:
# Überprüfe Datenqualität NACHHER
print("\n✅ Datenqualität NACH Bereinigung:")
print(f"  Gesamt-Zeilen: {len(df)}")
print(f"  NULL-Werte in oz_bez: {df['oz_bez'].isna().sum()}")
print(f"  Leere Strings in oz_bez: {(df['oz_bez'] == '').sum()}")
print(f"  Nur Whitespace in oz_bez: {df['oz_bez'].apply(is_empty_or_whitespace).sum()}")

if df['oz_bez'].apply(is_empty_or_whitespace).sum() == 0:
    print(f"  🎉 Keine ungültigen Texte mehr!")

In [ ]:
# %% md
# ## 60/20/20 Train/Val/Test Split - TextID gruppiert

# %%
# Split mit TextID Gruppenbildung
print("\n🔀 Führe 60/20/20 Split durch (TextID-Gruppen bleiben zusammen)...")

from sklearn.model_selection import train_test_split

# Prüfe ob textid Spalte existiert
if 'textid' not in df.columns:
    print("⚠️  'textid' Spalte nicht gefunden, nutze einfachen Split")
    train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
else:
    # Gruppiere nach textid - nur unique TextIDs splitten
    print("  ℹ️  Gruppiere nach 'textid'...")
    unique_textids = df['textid'].dropna().unique()
    print(f"  ℹ️  Unique TextIDs: {len(unique_textids)}")

    # Split die TextID-Gruppen (nicht die einzelnen Rows)
    train_textids, temp_textids = train_test_split(
        unique_textids,
        test_size=0.4,
        random_state=42
    )
    val_textids, test_textids = train_test_split(
        temp_textids,
        test_size=0.5,
        random_state=42
    )

    # Weise Rows basierend auf ihrer TextID den Sets zu
    train_df = df[df['textid'].isin(train_textids)].copy()
    val_df = df[df['textid'].isin(val_textids)].copy()
    test_df = df[df['textid'].isin(test_textids)].copy()

print(f"\n  ✓ Train-Set:      {len(train_df)} Zeilen ({len(train_df)/len(df)*100:.1f}%)")
print(f"  ✓ Validation-Set: {len(val_df)} Zeilen ({len(val_df)/len(df)*100:.1f}%)")
print(f"  ✓ Test-Set:       {len(test_df)} Zeilen ({len(test_df)/len(df)*100:.1f}%)")

# Kontrolle: Keine Overlap zwischen Sets
train_textids_set = set(train_df['textid'].dropna().unique())
val_textids_set = set(val_df['textid'].dropna().unique())
test_textids_set = set(test_df['textid'].dropna().unique())

overlap_train_val = len(train_textids_set & val_textids_set)
overlap_train_test = len(train_textids_set & test_textids_set)
overlap_val_test = len(val_textids_set & test_textids_set)

print(f"\n  🔍 Kontrolle (Overlaps sollten 0 sein):")
print(f"     Train ∩ Val:  {overlap_train_val}")
print(f"     Train ∩ Test: {overlap_train_test}")
print(f"     Val ∩ Test:   {overlap_val_test}")

if overlap_train_val == 0 and overlap_train_test == 0 and overlap_val_test == 0:
    print(f"  ✅ Keine TextID-Overlaps - Split ist sauber!")
else:
    print(f"  ⚠️  WARNUNG: Es gibt Overlaps!")

# %%

In [ ]:
# Speichern der Splits
print("\n💾 Speichere Splits...")

TRAIN_PATH = os.getenv("TRAIN_PATH", "data/train.csv")
VAL_PATH = os.getenv("VAL_PATH", "data/val.csv")
TEST_PATH = os.getenv("TEST_PATH", "data/test.csv")

# Erstelle Verzeichnis falls nötig
os.makedirs(os.path.dirname(TRAIN_PATH) or ".", exist_ok=True)

train_df.to_csv(TRAIN_PATH, index=False, encoding="utf-8")
val_df.to_csv(VAL_PATH, index=False, encoding="utf-8")
test_df.to_csv(TEST_PATH, index=False, encoding="utf-8")

print(f"  ✓ {TRAIN_PATH}")
print(f"  ✓ {VAL_PATH}")
print(f"  ✓ {TEST_PATH}")

print("\n✅ Splits erfolgreich erstellt und gespeichert!")
